In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
"""
ALNS for College Tour TSP in Istanbul (Final Integrated Version)
Features:
 - CSV Input Support (Kaggle Path)
 - ALNS + Simulated Annealing
 - Timer (Runtime calculation)
 - Excel Reporting (Detailed Routes + Summary)
 - Parameter Tuning Mode (Grid Search)
"""

import numpy as np
import pandas as pd
import random
import math
import csv
import time
from dataclasses import dataclass
from typing import List, Tuple
from collections import Counter

# ==============================
# 0. AYARLAR VE MOD SEÇİMİ
# ==============================

# DİKKAT: Tuning (Grid Search) yapmak istiyorsanız bunu True yapın.
# Tek seferlik normal çalıştırma ve rapor alma için False kalsın.
RUN_TUNING_MODE = False 

# DOSYA YOLU (Senin verdiğin CSV yolu)
DIST_CSV_PATH = "/kaggle/input/taksim-distance-matrixx/universite_distance_matrix_osrm_km_taksim.csv"
LOG_CSV_PATH = "outputs_alns_log.csv"
TUNING_CSV_PATH = "tuning_results.csv"
REPORT_EXCEL_PATH = "cozum_raporu.xlsx"

# "BÜTÇE" VE KISIT AYARLARI
N_DAYS = 20               # Toplam gün sayısı
AVG_SPEED = 30_000 / 60   # m/dakika (~30 km/h varsayımı)
SERVICE_TIME = 120        # Her üniversitede bekleme süresi (dakika)
MAX_DAILY_TIME = 8 * 60   # Günlük maksimum süre (dakika)
MAX_DAILY_DIST = 250_000  # Günlük maksimum mesafe (metre)

# CEZALAR
PRIORITY_PENALTY = 10_000    # Öncelik ihlali
LATE_TIME_PENALTY = 100      # Time window ihlali
TIME_OVER_PENALTY = 200      # Günlük zaman aşımı
DIST_OVER_PENALTY = 0.1      # Günlük mesafe aşımı
BIG_PENALTY = 100_000        # Hard constraint ihlali

# ALNS VARSAYILAN PARAMETRELERİ
INITIAL_T = 1000.0           # Başlangıç sıcaklığı
ALPHA_T = 0.99               # Soğuma katsayısı
MAX_ITER = 3000              # İterasyon sayısı

# ==============================
# 1. DISTANCE MATRIX OKUMA (CSV)
# ==============================

def load_distance_matrix_csv(path: str):
    """
    CSV formatı için dayanıklı okuyucu.
    Kaggle/TR formatları için hem ';' hem ',' ayırıcılarını dener.
    """
    try:
        # 1) Önce noktalı virgül (TR format) dene
        df = pd.read_csv(path, sep=';', decimal=',')
        df = df.dropna(axis=1, how='all')

        # 2) Eğer tek kolon geldiyse muhtemelen virgülle ayrılmıştır
        if df.shape[1] == 1:
            df = pd.read_csv(path, sep=',', decimal='.')
            df = df.dropna(axis=1, how='all')

        names = df.iloc[:, 0].astype(str).tolist()
        dist = df.iloc[:, 1:].to_numpy(dtype=float)

        print(f"[IO] Dosya yüklendi: {path}")
        print(f"[IO] Matris boyutu: {dist.shape}")
        return names, dist
    except Exception as e:
        print(f"[HATA] CSV okunamadı: {e}")
        # Hata durumunda boş döndür (kodun patlamaması için kontrol edilmeli)
        return [], np.zeros((1,1))

# Veriyi Yükle
UNIV_NAMES, DIST_MATRIX = load_distance_matrix_csv(DIST_CSV_PATH)
NUM_NODES = len(UNIV_NAMES)

# DEPOT: Taksim (Genelde listenin sonundadır, 59 satırsa index 58)
DEPOT = NUM_NODES - 1 

# BİRİM DÖNÜŞÜMÜ: Gelen veri KM ise Metreye çevir
DIST_MATRIX = DIST_MATRIX * 1000.0 

# ==============================
# 2. EK VERİLER
# ==============================

# (C) TIME WINDOWS: [0, 6 saat]
time_windows = np.zeros((NUM_NODES, 2))
time_windows[:, 0] = 0
time_windows[:, 1] = 6 * 60 

# (F) PRIORITY SET
priority = np.zeros(NUM_NODES, dtype=int)
# İlgili indexler (Veri setine göre değişebilir, örnek set)
prio_indices = [7, 12, 29, 37, 40, 41, 45, 47, 49, 50, 52, 54, 57]
for idx in prio_indices:
    if idx < NUM_NODES:
        priority[idx] = 1

MAX_GLOBAL_POSITION_FOR_PRIORITY = 19 

daily_time_limits = np.array([MAX_DAILY_TIME] * N_DAYS)
daily_dist_limits = np.array([MAX_DAILY_DIST] * N_DAYS)

# ==============================
# 3. ÇÖZÜM TEMSİLİ
# ==============================

@dataclass
class Solution:
    routes: List[List[int]]   # routes[d] = depot hariç node listesi
    cost: float = None
    feasible: bool = False

def build_full_route(day_route: List[int]) -> List[int]:
    return [DEPOT] + day_route + [DEPOT]

# ==============================
# 4. DEĞERLENDİRME (COST FUNCTION)
# ==============================

def evaluate_solution(sol: Solution) -> Solution:
    penalty = 0.0
    hard_penalty = 0.0
    total_dist = 0.0

    # (A) Ziyaret Sayısı Kontrolü
    visit_counts = np.zeros(NUM_NODES, dtype=int)
    for d in range(N_DAYS):
        for node in sol.routes[d]:
            visit_counts[node] += 1

    for i in range(NUM_NODES):
        if i == DEPOT: continue
        if visit_counts[i] == 0:
            penalty += BIG_PENALTY
            hard_penalty += BIG_PENALTY
        elif visit_counts[i] > 1:
            extra = BIG_PENALTY * (visit_counts[i] - 1)
            penalty += extra
            hard_penalty += extra

    # Global Sıralama (Priority için)
    global_order = []
    for d in range(N_DAYS):
        global_order.extend(sol.routes[d])

    # Gün Bazlı Kontroller
    for d in range(N_DAYS):
        route = sol.routes[d]
        full = build_full_route(route)
        
        day_dist = 0.0
        current_time = 0.0
        
        indeg = np.zeros(NUM_NODES, dtype=int)
        outdeg = np.zeros(NUM_NODES, dtype=int)

        for i in range(len(full) - 1):
            u = full[i]
            v = full[i + 1]
            
            dist_uv = DIST_MATRIX[u, v]
            day_dist += dist_uv
            travel_time = dist_uv / AVG_SPEED
            current_time += travel_time
            
            # Time Window & Service Time
            if v != DEPOT:
                a_i, b_i = time_windows[v]
                if current_time < a_i:
                    current_time = a_i
                current_time += SERVICE_TIME
                
                if current_time > b_i:
                    penalty += (current_time - b_i) * LATE_TIME_PENALTY
            
            outdeg[u] += 1
            indeg[v] += 1

        total_dist += day_dist
        day_time = current_time

        # (B) Flow Balance (Basit kontrol)
        for i in range(NUM_NODES):
            if i == DEPOT: continue
            if indeg[i] > 1 or outdeg[i] > 1:
                penalty += BIG_PENALTY
                hard_penalty += BIG_PENALTY

        # (D) Günlük Limitler
        if day_time > daily_time_limits[d]:
            penalty += (day_time - daily_time_limits[d]) * TIME_OVER_PENALTY
        if day_dist > daily_dist_limits[d]:
            penalty += (day_dist - daily_dist_limits[d]) * DIST_OVER_PENALTY

        # (E) Gün içi tekrar
        visited_nodes = [x for x in full if x != DEPOT]
        if len(visited_nodes) != len(set(visited_nodes)):
            penalty += BIG_PENALTY
            hard_penalty += BIG_PENALTY

    # (F) Priority Cezası
    for pos, node in enumerate(global_order):
        if priority[node] == 1 and pos > MAX_GLOBAL_POSITION_FOR_PRIORITY:
            penalty += PRIORITY_PENALTY

    sol.cost = total_dist + penalty
    sol.feasible = (hard_penalty == 0.0)
    return sol

def cost_breakdown(sol: Solution):
    """Konsola detaylı maliyet raporu basar"""
    print("\n=== COST BREAKDOWN ===")
    print(f"Total Cost: {sol.cost:.1f}")
    print(f"Feasible: {sol.feasible}")
    # (Detaylı print işlemleri evaluate_solution içindeki mantığın aynısıdır,
    # yer kaplamaması için burada özet geçiyorum, algoritma evaluate fonksiyonunu kullanır)
    print("=== END COST BREAKDOWN ===\n")

# ==============================
# 5. ALNS OPERATÖRLERİ
# ==============================

def construct_initial_solution() -> Solution:
    nodes = list(range(NUM_NODES))
    if DEPOT in nodes: nodes.remove(DEPOT)
    random.shuffle(nodes)

    routes = [[] for _ in range(N_DAYS)]
    for idx, node in enumerate(nodes):
        routes[idx % N_DAYS].append(node)
    
    return evaluate_solution(Solution(routes=routes))

def random_removal(sol: Solution, q: int) -> Tuple[Solution, List[int]]:
    routes = [r[:] for r in sol.routes]
    all_nodes = [(d, i, node) for d, r in enumerate(routes) for i, node in enumerate(r)]
    
    if not all_nodes: return evaluate_solution(Solution(routes=routes)), []
    
    q = min(q, len(all_nodes))
    chosen = random.sample(all_nodes, q)
    chosen.sort(key=lambda x: (x[0], -x[1])) # Sondan başa silmek için sırala
    
    removed = []
    for d, i, node in chosen:
        del routes[d][i]
        removed.append(node)
        
    return evaluate_solution(Solution(routes=routes)), removed

def greedy_insert(sol: Solution, request_bank: List[int]) -> Solution:
    current = evaluate_solution(Solution(routes=[r[:] for r in sol.routes]))
    
    for u in request_bank:
        best_cost = math.inf
        best_d, best_p = 0, 0
        
        for d in range(N_DAYS):
            r_len = len(current.routes[d])
            for p in range(r_len + 1):
                cand_routes = [r[:] for r in current.routes]
                cand_routes[d].insert(p, u)
                cand_sol = evaluate_solution(Solution(routes=cand_routes))
                
                if cand_sol.cost < best_cost:
                    best_cost = cand_sol.cost
                    best_d, best_p = d, p
        
        current.routes[best_d].insert(best_p, u)
        current = evaluate_solution(current)
        
    return current

# ==============================
# 6. LOCAL SEARCH (2-OPT)
# ==============================

def route_length(route: List[int]) -> float:
    full = build_full_route(route)
    total = 0.0
    for i in range(len(full)-1):
        total += DIST_MATRIX[full[i], full[i+1]]
    return total

def two_opt_day(route: List[int]) -> List[int]:
    improved = True
    best_r = route[:]
    best_l = route_length(best_r)
    
    while improved:
        improved = False
        for i in range(len(best_r) - 1):
            for j in range(i+2, len(best_r)):
                new_r = best_r[:]
                new_r[i+1:j+1] = reversed(new_r[i+1:j+1])
                new_l = route_length(new_r)
                if new_l + 1e-6 < best_l:
                    best_l = new_l
                    best_r = new_r
                    improved = True
                    break
            if improved: break
    return best_r

def apply_two_opt(sol: Solution) -> Solution:
    new_routes = [two_opt_day(r) for r in sol.routes]
    return evaluate_solution(Solution(routes=new_routes))

# ==============================
# 7. ALNS ANA ALGORİTMA
# ==============================

def alns(initial_t=INITIAL_T, alpha_t=ALPHA_T, log_to_file=True) -> Solution:
    S_curr = construct_initial_solution()
    S_best = S_curr
    T = initial_t
    history = []
    
    for it in range(1, MAX_ITER + 1):
        if T < 1e-3: break
        
        # Destroy & Repair
        q = max(1, (NUM_NODES - 1) // 10)
        S_tmp, bank = random_removal(S_curr, q)
        S_new = greedy_insert(S_tmp, bank)
        
        # Local Search
        S_new = apply_two_opt(S_new)
        
        # Acceptance (Simulated Annealing)
        delta = S_new.cost - S_curr.cost
        if delta < 0 or random.random() < math.exp(-delta / T):
            S_curr = S_new
            
        if S_curr.cost < S_best.cost:
            S_best = S_curr
            
        history.append((it, T, S_curr.cost, S_best.cost, int(S_best.feasible)))
        T *= alpha_t
        
        if log_to_file and it % 100 == 0:
            print(f"Iter {it}, T={T:.2f}, Best={S_best.cost:.1f}, Feas={S_best.feasible}")
            
    if log_to_file:
        with open(LOG_CSV_PATH, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["iter", "T", "curr", "best", "feas"])
            w.writerows(history)
            
    return S_best

# ==============================
# 8. TUNING & EXCEL EXPORT
# ==============================

def run_grid_tuning():
    print("\n=== TUNING MODE ===")
    init_opts = [300.0, 1000.0, 3000.0]
    alpha_opts = [0.95, 0.98, 0.99]
    repeats = 10
    results = []
    
    count = 0
    total = len(init_opts) * len(alpha_opts) * repeats
    
    for t in init_opts:
        for a in alpha_opts:
            for r in range(repeats):
                count += 1
                seed = 1000 + count
                random.seed(seed)
                np.random.seed(seed)
                
                print(f"[TUNING] {count}/{total}: T={t}, Alpha={a}")
                # Hız için log_to_file=False
                best = alns(initial_t=t, alpha_t=a, log_to_file=False)
                results.append([count, t, a, r+1, seed, best.cost, best.feasible])
                
    with open(TUNING_CSV_PATH, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["ID", "Init_T", "Alpha", "Rep", "Seed", "Cost", "Feas"])
        w.writerows(results)
    print(f"\nTuning results saved to {TUNING_CSV_PATH}")

def save_final_excel(sol: Solution, runtime_sec: float, filename: str):
    """Rotaları ve özeti Excel'e sekmeli yazar"""
    rows = []
    for d, r in enumerate(sol.routes):
        for i, node in enumerate(r):
            rows.append({
                "Day": d+1,
                "Order": i+1,
                "University": UNIV_NAMES[node],
                "ID": node,
                "Priority": "Yes" if priority[node] else "No"
            })
    
    df_routes = pd.DataFrame(rows)
    df_summ = pd.DataFrame([{
        "Total Cost": sol.cost,
        "Feasible": sol.feasible,
        "Runtime (sec)": runtime_sec,
        "Algorithm": "ALNS + SA"
    }])
    
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df_routes.to_excel(writer, sheet_name="Routes", index=False)
            df_summ.to_excel(writer, sheet_name="Summary", index=False)
        print(f"\n[EXCEL] Rapor kaydedildi: {filename}")
    except Exception as e:
        print(f"\n[HATA] Excel yazılamadı (openpyxl yüklü mü?): {e}")

# ==============================
# 9. MAIN EXECUTION
# ==============================

if __name__ == "__main__":
    
    if RUN_TUNING_MODE:
        run_grid_tuning()
        
    else:
        print("=== NORMAL MOD BAŞLATILIYOR ===")
        
        # 1. Başlangıç
        start_time = time.perf_counter()
        
        # 2. Çalıştır
        best_solution = alns(initial_t=INITIAL_T, alpha_t=ALPHA_T, log_to_file=True)
        
        # 3. Bitiş
        end_time = time.perf_counter()
        runtime = end_time - start_time
        
        # 4. Sonuçları Ekrana Bas
        print("\n=== EN İYİ ÇÖZÜM ===")
        print(f"Maliyet: {best_solution.cost:.1f}")
        print(f"Fizibilite: {best_solution.feasible}")
        print(f"Süre: {runtime:.2f} saniye")
        
        for d, r in enumerate(best_solution.routes):
            uni_list = [UNIV_NAMES[u] for u in r]
            print(f"Gün {d+1}: {uni_list}")
            
        cost_breakdown(best_solution)
        
        # 5. Excel'e Kaydet
        save_final_excel(best_solution, runtime, REPORT_EXCEL_PATH)

[IO] Dosya yüklendi: /kaggle/input/taksim-distance-matrixx/universite_distance_matrix_osrm_km_taksim.csv
[IO] Matris boyutu: (59, 59)
=== NORMAL MOD BAŞLATILIYOR ===
Iter 100, T=366.03, Best=1414680.2, Feas=True
Iter 200, T=133.98, Best=1357241.6, Feas=True
Iter 300, T=49.04, Best=1342382.6, Feas=True
Iter 400, T=17.95, Best=1319907.2, Feas=True
Iter 500, T=6.57, Best=1312718.1, Feas=True
Iter 600, T=2.41, Best=1303978.3, Feas=True
Iter 700, T=0.88, Best=1296929.9, Feas=True
Iter 800, T=0.32, Best=1296477.8, Feas=True
Iter 900, T=0.12, Best=1296477.8, Feas=True
Iter 1000, T=0.04, Best=1296330.5, Feas=True
Iter 1100, T=0.02, Best=1294684.2, Feas=True
Iter 1200, T=0.01, Best=1294684.2, Feas=True
Iter 1300, T=0.00, Best=1291266.5, Feas=True

=== EN İYİ ÇÖZÜM ===
Maliyet: 1291266.5
Fizibilite: True
Süre: 339.57 saniye
Gün 1: ['Acıbadem Mehmet Ali Aydınlar Üniversitesi ', 'YEDİTEPE ÜNİVERSİTESİ', 'MALTEPE ÜNİVERSİTESİ']
Gün 2: ['İSTANBUL AREL ÜNİVERSİTESİ', 'İSTANBUL RUMELİ ÜNİVERSİTESİ']
G

In [2]:
"""
ALNS for College Tour TSP in Istanbul (Fixed + Logging + Tuning)

- Multi-day routes starting/ending at Taksim (depot)
- Distance matrix from Kaggle CSV
- Directed/asymmetric matrix supported

Constraints:
  (A) Each university visited exactly once (except depot) -> HARD
  (B) Flow balance (kept as HARD, but optional)
  (C) Time windows -> SOFT
  (D) Daily travel distance / time limits -> SOFT
  (E) No repeated nodes within a day -> HARD
  (F) Priority universities early -> SOFT

ALNS:
  - Destroy: random removal
  - Repair : greedy insert
  - Local search: 2-opt per day
  - Acceptance: Simulated Annealing

Logging:
  - Writes outputs_alns_log.csv with per-iteration objective values
"""

import numpy as np
import pandas as pd
import random
import math
import csv
from dataclasses import dataclass
from typing import List, Tuple
from collections import Counter

# ==============================
# 0. PARAMETERS / "BÜTÇE" AYARLARI
# ==============================

N_DAYS = 20               # toplam gün sayısı
AVG_SPEED = 30_000 / 60   # m/dakika, ~30 km/h varsayımı
SERVICE_TIME = 120        # her üniversitede 120 dk kalma süresi
MAX_DAILY_TIME = 8 * 60   # günlük maksimum 8 saat (dakika)
MAX_DAILY_DIST = 250_000  # günlük maksimum 250 km (metre)

PRIORITY_PENALTY = 10_000    # öncelik ihlali cezası
LATE_TIME_PENALTY = 100      # time window aşımı cezası
TIME_OVER_PENALTY = 200      # günlük zaman limiti aşımı cezası
DIST_OVER_PENALTY = 0.1      # günlük mesafe limiti aşımı cezası (metre başına)
BIG_PENALTY = 100_000        # sert ihlaller için büyük ceza

INITIAL_T = 1000.0           # başlangıç sıcaklığı (SA)
ALPHA_T = 0.99               # soğuma oranı
MAX_ITER = 3000              # maksimum iterasyon

# Logging output (Kaggle working dir)
LOG_CSV_PATH = "outputs_alns_log.csv"
LOG_EVERY = 1  # 1 -> her iter; 10 -> 10 iterde bir logla

# PARAMETER TUNING
RUN_TUNING = False  # True yaparsan grid search çalışır ve tuning_results.csv üretir

# ==============================
# 1. DISTANCE MATRIX OKUMA (CSV)
# ==============================

def load_distance_matrix_csv(path: str):
    """
    CSV formatı için dayanıklı okuyucu.
    TR Excel export senaryosu:
      - sep=';'
      - decimal=','
      - first column = names
      - rest = numeric matrix
    """
    # 1) TR formatı dene
    df = pd.read_csv(path, sep=';', decimal=',')
    df = df.dropna(axis=1, how='all')

    # 2) tek kolon geldiyse standart csv dene
    if df.shape[1] == 1:
        df = pd.read_csv(path, sep=',', decimal='.')
        df = df.dropna(axis=1, how='all')

    names = df.iloc[:, 0].astype(str).tolist()
    dist = df.iloc[:, 1:].to_numpy(dtype=float)

    print("Distance matrix shape:", dist.shape)
    return names, dist


# Kaggle input path (SENİN DOSYAN)
DIST_CSV_PATH = "/kaggle/input/taksim-distance-matrixx/universite_distance_matrix_osrm_km_taksim.csv"

UNIV_NAMES, DIST_MATRIX = load_distance_matrix_csv(DIST_CSV_PATH)
NUM_NODES = len(UNIV_NAMES)

# DEPOT index (senin case: 59 node -> 0..58, depot=58)
DEPOT = 58

# ---- ÖNEMLİ: matrix birimi ----
# Dosya adında "km" geçiyor: eğer gerçekten KM ise metreye çevir.
# Eğer zaten metre ise bunu False yap.
DIST_IS_IN_KM = True
if DIST_IS_IN_KM:
    DIST_MATRIX = DIST_MATRIX * 1000.0  # km -> m

# ==============================
# 2. EK VERİLER
# ==============================

# (C) TIME WINDOWS: her node için [a_i, b_i] (dakika)
time_windows = np.zeros((NUM_NODES, 2))
time_windows[:, 0] = 0
time_windows[:, 1] = 6 * 60  # 6 saat (istersen 8*60 yap)

# (F) PRIORITY SET: öncelikli üniversiteler
priority = np.zeros(NUM_NODES, dtype=int)
for idx in [7, 12, 29, 37, 40, 41, 45, 47, 49, 50, 52, 54, 57]:
    if idx < NUM_NODES:
        priority[idx] = 1

MAX_GLOBAL_POSITION_FOR_PRIORITY = 19  # ilk 20 ziyaret

daily_time_limits = np.array([MAX_DAILY_TIME] * N_DAYS)
daily_dist_limits = np.array([MAX_DAILY_DIST] * N_DAYS)

# ==============================
# 3. SOLUTION REPRESENTATION
# ==============================

@dataclass
class Solution:
    routes: List[List[int]]   # routes[d] = depot hariç node listesi
    cost: float = None
    feasible: bool = False


def build_full_route(day_route: List[int]) -> List[int]:
    return [DEPOT] + day_route + [DEPOT]

# ==============================
# 4. EVALUATION (COST + FEASIBILITY)
# ==============================

def evaluate_solution(sol: Solution) -> Solution:
    penalty = 0.0
    hard_penalty = 0.0
    total_dist = 0.0

    # (A) Each university visited exactly once (depot hariç)
    visit_counts = np.zeros(NUM_NODES, dtype=int)
    for d in range(N_DAYS):
        for node in sol.routes[d]:
            visit_counts[node] += 1

    for i in range(NUM_NODES):
        if i == DEPOT:
            continue
        if visit_counts[i] == 0:
            penalty += BIG_PENALTY
            hard_penalty += BIG_PENALTY
        elif visit_counts[i] > 1:
            extra = BIG_PENALTY * (visit_counts[i] - 1)
            penalty += extra
            hard_penalty += extra

    # Global visiting order (priority için)
    global_order: List[int] = []
    for d in range(N_DAYS):
        global_order.extend(sol.routes[d])

    # Day-by-day
    for d in range(N_DAYS):
        route = sol.routes[d]
        full = build_full_route(route)

        day_dist = 0.0
        current_time = 0.0

        # (B) flow balance
        indeg = np.zeros(NUM_NODES, dtype=int)
        outdeg = np.zeros(NUM_NODES, dtype=int)

        for i in range(len(full) - 1):
            u = full[i]
            v = full[i + 1]

            dist_uv = DIST_MATRIX[u, v]
            day_dist += dist_uv

            travel_time = dist_uv / AVG_SPEED
            current_time += travel_time

            # TW + service only for non-depot
            if v != DEPOT:
                a_i, b_i = time_windows[v]
                if current_time < a_i:
                    current_time = a_i  # wait if early
                current_time += SERVICE_TIME
                if current_time > b_i:
                    penalty += (current_time - b_i) * LATE_TIME_PENALTY

            indeg[v] += 1
            outdeg[u] += 1

        day_time = current_time
        total_dist += day_dist

        # (B) Flow balance -> HARD (keep)
        for i in range(NUM_NODES):
            if i == DEPOT:
                continue
            if indeg[i] > 1 or outdeg[i] > 1:
                penalty += BIG_PENALTY
                hard_penalty += BIG_PENALTY

        # (D) Daily limits -> SOFT
        if day_time > daily_time_limits[d]:
            penalty += (day_time - daily_time_limits[d]) * TIME_OVER_PENALTY
        if day_dist > daily_dist_limits[d]:
            penalty += (day_dist - daily_dist_limits[d]) * DIST_OVER_PENALTY

        # (E) No repeated nodes inside a day -> HARD
        visited = [x for x in full if x != DEPOT]
        if len(visited) != len(set(visited)):
            penalty += BIG_PENALTY
            hard_penalty += BIG_PENALTY

    # (F) Priority -> SOFT
    for pos, node in enumerate(global_order):
        if priority[node] == 1 and pos > MAX_GLOBAL_POSITION_FOR_PRIORITY:
            penalty += PRIORITY_PENALTY

    sol.cost = total_dist + penalty
    sol.feasible = (hard_penalty == 0.0)
    return sol

# ==============================
# 4.1 COST BREAKDOWN
# ==============================

def cost_breakdown(sol: Solution):
    print("\n=== COST BREAKDOWN ===")

    base_distance = 0.0

    penA_visit = 0.0
    penB_flow = 0.0
    penE_subtour = 0.0

    penC_tw = 0.0
    penD_time = 0.0
    penD_dist = 0.0
    penF_prio = 0.0

    total_late_minutes = 0.0
    total_overtime_minutes = 0.0
    total_overdist_meters = 0.0
    prio_violations = 0

    # (A)
    visit_counts = np.zeros(NUM_NODES, dtype=int)
    for route in sol.routes:
        for node in route:
            visit_counts[node] += 1

    for i in range(NUM_NODES):
        if i == DEPOT:
            continue
        if visit_counts[i] == 0:
            penA_visit += BIG_PENALTY
        elif visit_counts[i] > 1:
            penA_visit += BIG_PENALTY * (visit_counts[i] - 1)

    # global order (F)
    global_order: List[int] = []
    for route in sol.routes:
        global_order.extend(route)

    # day-by-day
    for d, route in enumerate(sol.routes):
        full = build_full_route(route)
        day_dist = 0.0
        current_time = 0.0

        indeg = np.zeros(NUM_NODES, dtype=int)
        outdeg = np.zeros(NUM_NODES, dtype=int)

        for i in range(len(full) - 1):
            u = full[i]
            v = full[i + 1]

            dist_uv = DIST_MATRIX[u, v]
            day_dist += dist_uv

            travel_time = dist_uv / AVG_SPEED
            current_time += travel_time

            if v != DEPOT:
                a_i, b_i = time_windows[v]
                if current_time < a_i:
                    current_time = a_i
                current_time += SERVICE_TIME
                if current_time > b_i:
                    lateness = current_time - b_i
                    total_late_minutes += lateness
                    penC_tw += lateness * LATE_TIME_PENALTY

            outdeg[u] += 1
            indeg[v] += 1

        base_distance += day_dist

        # (B)
        for i in range(NUM_NODES):
            if i == DEPOT:
                continue
            if indeg[i] > 1 or outdeg[i] > 1:
                penB_flow += BIG_PENALTY

        # (D)
        day_time = current_time
        if day_time > daily_time_limits[d]:
            over_t = day_time - daily_time_limits[d]
            total_overtime_minutes += over_t
            penD_time += over_t * TIME_OVER_PENALTY

        if day_dist > daily_dist_limits[d]:
            over_d = day_dist - daily_dist_limits[d]
            total_overdist_meters += over_d
            penD_dist += over_d * DIST_OVER_PENALTY

        # (E)
        visited = [x for x in full if x != DEPOT]
        if len(visited) != len(set(visited)):
            penE_subtour += BIG_PENALTY

    # (F)
    for pos, node in enumerate(global_order):
        if priority[node] == 1 and pos > MAX_GLOBAL_POSITION_FOR_PRIORITY:
            prio_violations += 1
            penF_prio += PRIORITY_PENALTY

    hard_total = penA_visit + penB_flow + penE_subtour
    soft_total = penC_tw + penD_time + penD_dist + penF_prio
    total_cost = base_distance + hard_total + soft_total

    print(f"Base distance: {base_distance/1000:.3f} km ({base_distance:.1f} m)\n")
    print("HARD penalties:")
    print(f"  (A) Visit-count:  {penA_visit:.1f}")
    print(f"  (B) Flow-balance: {penB_flow:.1f}")
    print(f"  (E) Repeat-day:   {penE_subtour:.1f}")
    print(f"  -> HARD total:    {hard_total:.1f}\n")

    print("SOFT penalties:")
    print(f"  (C) Time window:  {penC_tw:.1f} (lateness={total_late_minutes:.1f} min × {LATE_TIME_PENALTY})")
    print(f"  (D) Time limit:   {penD_time:.1f} (overtime={total_overtime_minutes:.1f} min × {TIME_OVER_PENALTY})")
    print(f"  (D) Dist limit:   {penD_dist:.1f} (overdist={total_overdist_meters:.1f} m × {DIST_OVER_PENALTY})")
    print(f"  (F) Priority:     {penF_prio:.1f} (viol={prio_violations} × {PRIORITY_PENALTY})")
    print(f"  -> SOFT total:    {soft_total:.1f}\n")

    print(f"TOTAL COST (recalc): {total_cost:.1f}")
    print(f"sol.cost:            {sol.cost:.1f}")
    print("=== END COST BREAKDOWN ===\n")

# ==============================
# 5. INITIAL SOLUTION
# ==============================

def construct_initial_solution() -> Solution:
    nodes = list(range(NUM_NODES))
    nodes.remove(DEPOT)
    random.shuffle(nodes)

    routes: List[List[int]] = [[] for _ in range(N_DAYS)]
    for idx, node in enumerate(nodes):
        routes[idx % N_DAYS].append(node)

    return evaluate_solution(Solution(routes=routes))

# ==============================
# 5.1 HARD CONSTRAINT DEBUG
# ==============================

def check_hard_violations(sol: Solution):
    print("\n=== HARD CONSTRAINT CHECK ===")

    visit_counts = np.zeros(NUM_NODES, dtype=int)
    for route in sol.routes:
        for node in route:
            visit_counts[node] += 1

    missing = [i for i in range(NUM_NODES) if i != DEPOT and visit_counts[i] == 0]
    multi = [i for i in range(NUM_NODES) if i != DEPOT and visit_counts[i] > 1]

    if not missing and not multi:
        print("(A) Visit count: OK")
    else:
        print("(A) Visit count violations:")
        for i in missing:
            print(f"  - Missing: {i} {UNIV_NAMES[i]}")
        for i in multi:
            print(f"  - Multi:   {i} {UNIV_NAMES[i]} count={visit_counts[i]}")

    any_repeat = False
    for d, route in enumerate(sol.routes):
        full = build_full_route(route)
        visited = [x for x in full if x != DEPOT]
        if len(visited) != len(set(visited)):
            any_repeat = True
            cnt = Counter(visited)
            rep = [i for i, c in cnt.items() if c > 1]
            print(f"(E) Day {d+1} repeats:")
            for i in rep:
                print(f"  - {i} {UNIV_NAMES[i]} count={cnt[i]}")

    if not any_repeat:
        print("(E) Day repeats: OK")

    print("=== END HARD CONSTRAINT CHECK ===\n")

# ==============================
# 6. DESTROY / REPAIR
# ==============================

def random_removal(sol: Solution, q: int) -> Tuple[Solution, List[int]]:
    routes = [r[:] for r in sol.routes]
    all_nodes = [(d, i, node) for d, r in enumerate(routes) for i, node in enumerate(r)]
    if len(all_nodes) == 0:
        return evaluate_solution(Solution(routes=routes)), []

    q = min(q, len(all_nodes))
    chosen = random.sample(all_nodes, q)

    removed_nodes: List[int] = []
    chosen_sorted = sorted(chosen, key=lambda x: (x[0], -x[1]))
    for d, i, node in chosen_sorted:
        del routes[d][i]
        removed_nodes.append(node)

    return evaluate_solution(Solution(routes=routes)), removed_nodes


def greedy_insert(sol: Solution, request_bank: List[int]) -> Solution:
    current = evaluate_solution(Solution(routes=[r[:] for r in sol.routes]))

    for u in request_bank:
        best_cost = math.inf
        best_d, best_pos = 0, 0

        for d in range(N_DAYS):
            route = current.routes[d]
            for k in range(len(route) + 1):
                candidate_routes = [r[:] for r in current.routes]
                candidate_routes[d].insert(k, u)
                cand = evaluate_solution(Solution(routes=candidate_routes))

                if cand.cost < best_cost:
                    best_cost = cand.cost
                    best_d, best_pos = d, k

        current.routes[best_d].insert(best_pos, u)
        current = evaluate_solution(current)

    return current

# ==============================
# 7. LOCAL SEARCH: 2-OPT
# ==============================

def route_length(route: List[int]) -> float:
    full = build_full_route(route)
    total = 0.0
    for i in range(len(full) - 1):
        total += DIST_MATRIX[full[i], full[i + 1]]
    return total


def two_opt_day(route: List[int]) -> List[int]:
    improved = True
    best_route = route[:]
    best_len = route_length(best_route)

    while improved:
        improved = False
        for i in range(len(best_route) - 1):
            for j in range(i + 2, len(best_route)):
                new_route = best_route[:]
                new_route[i + 1:j + 1] = reversed(new_route[i + 1:j + 1])
                new_len = route_length(new_route)
                if new_len + 1e-6 < best_len:
                    best_len = new_len
                    best_route = new_route
                    improved = True
                    break
            if improved:
                break

    return best_route


def apply_two_opt(sol: Solution) -> Solution:
    new_routes = [two_opt_day(r) for r in sol.routes]
    return evaluate_solution(Solution(routes=new_routes))

# ==============================
# 8. ALNS + LOGGING
# ==============================

def alns(verbose=True, write_log=True) -> Solution:
    S_current = construct_initial_solution()
    S_best = S_current

    T = INITIAL_T
    it = 0
    history = []

    while it < MAX_ITER and T > 1e-3:
        it += 1

        q = max(1, (NUM_NODES - 1) // 10)  # depot hariç ~%10

        S_temp, request_bank = random_removal(S_current, q=q)
        S_new = greedy_insert(S_temp, request_bank)
        S_new = apply_two_opt(S_new)

        deltaE = S_new.cost - S_current.cost
        if deltaE < 0 or random.random() < math.exp(-deltaE / T):
            S_current = S_new

        if S_current.cost < S_best.cost:
            S_best = S_current

        if it % LOG_EVERY == 0:
            history.append((it, T, S_current.cost, S_best.cost, int(S_best.feasible)))

        T *= ALPHA_T

        if verbose and it % 100 == 0:
            print(f"Iter {it}, best cost={S_best.cost:.1f}, feasible={S_best.feasible}")

    if write_log:
        with open(LOG_CSV_PATH, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["iter", "T", "current_cost", "best_cost", "feasible_best"])
            w.writerows(history)
        if verbose:
            print(f"\n[LOG] Iteration history written to: {LOG_CSV_PATH}")

    return S_best

# ==============================
# 9. PARAMETER TUNING (GRID SEARCH)
# ==============================

import time

def run_one(init_T, alpha_T, seed):
    m.INITIAL_T = init_T
    m.ALPHA_T = alpha_T

    random.seed(seed)
    np.random.seed(seed)

    start = time.perf_counter()
    best = m.alns()
    end = time.perf_counter()

    runtime = end - start
    return best.cost, int(best.feasible), runtime



def grid_tuning():
    init_list = [300.0, 1000.0, 3000.0]
    alpha_list = [0.95, 0.98, 0.99]
    repeats = 3

    rows = []
    exp_id = 0

    for init_T in init_list:
        for alpha_T in alpha_list:
            for r in range(repeats):
                exp_id += 1
                seed = 1000 + exp_id
                print(f"[TUNING] Exp {exp_id}: INITIAL_T={init_T}, ALPHA_T={alpha_T}, seed={seed}")
                cost, feas = run_one_experiment(init_T, alpha_T, seed=seed)
                rows.append([exp_id, init_T, alpha_T, r+1, seed, cost, feas])

    out_path = "tuning_results.csv"
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["exp_id", "INITIAL_T", "ALPHA_T", "repeat", "seed", "final_best_cost", "feasible"])
        w.writerows(rows)

    print(f"[TUNING] Saved: {out_path}")

# ==============================
# 10. MAIN
# ==============================

if __name__ == "__main__":
    if RUN_TUNING:
        grid_tuning()
    else:
        init_sol = construct_initial_solution()
        print("Initial cost:", init_sol.cost, "feasible:", init_sol.feasible)
        check_hard_violations(init_sol)

        best = alns(verbose=True, write_log=True)

        print("\n=== BEST SOLUTION ===")
        print("Cost:", best.cost)
        print("Feasible:", best.feasible)

        for d, r in enumerate(best.routes):
            names = [UNIV_NAMES[i] for i in r]
            print(f"Day {d+1}: {names}")

        # Priority check
        global_order: List[int] = []
        for r in best.routes:
            global_order.extend(r)

        print("\n=== PRIORITY CHECK (positions) ===")
        for pos, node in enumerate(global_order):
            if priority[node] == 1:
                print(pos, UNIV_NAMES[node])

        cost_breakdown(best)
        
import time
import pandas as pd

if __name__ == "__main__":
    start_time = time.perf_counter()

    best = alns()

    end_time = time.perf_counter()
    runtime_sec = end_time - start_time

    # Excel'e yaz
    df = pd.DataFrame([{
        "total_runtime_seconds": runtime_sec,
        "total_runtime_minutes": runtime_sec / 60,
        "best_cost": best.cost,
        "feasible": best.feasible
    }])

    df.to_excel("runtime_summary.xlsx", index=False)

    print("\n=== RUNTIME INFO ===")
    print(f"Total runtime: {runtime_sec:.2f} sec ({runtime_sec/60:.2f} min)")
    print("Saved -> runtime_summary.xlsx")
# ==============================
# PARAMETER TUNING + RUNTIME
# ==============================

import time
import pandas as pd
import numpy as np
import random

def run_one(init_T, alpha_T, seed):
    global INITIAL_T, ALPHA_T

    old_init, old_alpha = INITIAL_T, ALPHA_T
    INITIAL_T = init_T
    ALPHA_T = alpha_T

    random.seed(seed)
    np.random.seed(seed)

    t0 = time.time()
    best = alns()
    runtime_sec = time.time() - t0

    INITIAL_T, ALPHA_T = old_init, old_alpha
    return float(best.cost), int(best.feasible), float(runtime_sec)


def grid_tuning_excel():
    init_list = [300.0, 1000.0, 3000.0]
    alpha_list = [0.95, 0.98, 0.99]
    repeats = 10

    rows = []
    exp_id = 0

    for init_T in init_list:
        for alpha_T in alpha_list:
            for r in range(repeats):
                exp_id += 1
                seed = 1000 + exp_id
                print(f"[TUNING] exp={exp_id} INITIAL_T={init_T} ALPHA_T={alpha_T} rep={r+1}")

                cost, feas, rt = run_one(init_T, alpha_T, seed)
                rows.append([exp_id, init_T, alpha_T, r+1, seed, cost, feas, rt])

    df = pd.DataFrame(
        rows,
        columns=[
            "exp_id",
            "INITIAL_T",
            "ALPHA_T",
            "repeat",
            "seed",
            "final_best_cost",
            "feasible",
            "runtime_sec"
        ]
    )

    df.to_excel("tuning_results.xlsx", index=False)
    print("\nSaved -> tuning_results.xlsx")
    
if __name__ == "__main__":
    grid_tuning_excel()




Distance matrix shape: (59, 59)
Initial cost: 2586667.2000000007 feasible: True

=== HARD CONSTRAINT CHECK ===
(A) Visit count: OK
(E) Day repeats: OK
=== END HARD CONSTRAINT CHECK ===

Iter 100, best cost=1312166.3, feasible=True
Iter 200, best cost=1288579.9, feasible=True
Iter 300, best cost=1271609.9, feasible=True
Iter 400, best cost=1267927.8, feasible=True
Iter 500, best cost=1265658.0, feasible=True
Iter 600, best cost=1265658.0, feasible=True
Iter 700, best cost=1262353.6, feasible=True
Iter 800, best cost=1252628.0, feasible=True
Iter 900, best cost=1250466.9, feasible=True
Iter 1000, best cost=1248242.6, feasible=True
Iter 1100, best cost=1248242.6, feasible=True
Iter 1200, best cost=1248242.6, feasible=True
Iter 1300, best cost=1248242.6, feasible=True

[LOG] Iteration history written to: outputs_alns_log.csv

=== BEST SOLUTION ===
Cost: 1248242.62
Feasible: True
Day 1: ['DEMİROĞLU BİLİM ÜNİVERSİTESİ', 'YILDIZ TEKNİK ÜNİVERSİTESİ', 'GALATASARAY ÜNİVERSİTESİ']
Day 2: ['İSTAN

In [3]:
"""
ALNS Istanbul College Tour TSP - CHAMPION MODE (10 Runs)
- Parametreler: T=3000, Alpha=0.99
- 10 kez çalışır, en iyisini seçer ve Excel'e kaydeder.
- Tuning (Grid Search) YOK.
"""

import numpy as np
import pandas as pd
import random
import math
import csv
import time
from dataclasses import dataclass
from typing import List, Tuple
from collections import Counter

# ==============================
# 0. SABİT AYARLAR
# ==============================

# EN İYİ PARAMETRELER (Tuning sonucu belirlendi)
INITIAL_T = 3000.0
ALPHA_T = 0.99
MAX_ITER = 3000 
REPEAT_COUNT = 10  # Kaç kez tekrar edip en iyisini seçecek?

# DOSYA YOLU (Senin verdiğin Kaggle yolu)
DIST_CSV_PATH = "/kaggle/input/taksim-distance-matrixx/universite_distance_matrix_osrm_km_taksim.csv"
REPORT_EXCEL_PATH = "cozum_raporu.xlsx"

# KISIT VE BÜTÇE AYARLARI
N_DAYS = 20               
AVG_SPEED = 30_000 / 60   # m/dakika (~30 km/h)
SERVICE_TIME = 120        # 120 dk bekleme
MAX_DAILY_TIME = 8 * 60   
MAX_DAILY_DIST = 250_000  

# CEZALAR
PRIORITY_PENALTY = 10_000    
LATE_TIME_PENALTY = 100      
TIME_OVER_PENALTY = 200      
DIST_OVER_PENALTY = 0.1      
BIG_PENALTY = 100_000        

# ==============================
# 1. VERİ OKUMA VE HAZIRLIK
# ==============================

def load_distance_matrix_csv(path: str):
    try:
        # TR format (;) veya STD format (,) kontrolü
        df = pd.read_csv(path, sep=';', decimal=',')
        df = df.dropna(axis=1, how='all')

        if df.shape[1] == 1:
            df = pd.read_csv(path, sep=',', decimal='.')
            df = df.dropna(axis=1, how='all')

        names = df.iloc[:, 0].astype(str).tolist()
        dist = df.iloc[:, 1:].to_numpy(dtype=float)
        
        print(f"[IO] Dosya okundu. Matris: {dist.shape}")
        return names, dist
    except Exception as e:
        print(f"[HATA] Dosya okunamadı: {e}")
        return [], np.zeros((1,1))

UNIV_NAMES, DIST_MATRIX = load_distance_matrix_csv(DIST_CSV_PATH)
NUM_NODES = len(UNIV_NAMES)
DEPOT = NUM_NODES - 1  # Taksim (Son satır varsayımı)

# KM -> Metre dönüşümü
DIST_MATRIX = DIST_MATRIX * 1000.0 

# Time Windows & Priority
time_windows = np.zeros((NUM_NODES, 2))
time_windows[:, 1] = 6 * 60 

priority = np.zeros(NUM_NODES, dtype=int)
prio_indices = [7, 12, 29, 37, 40, 41, 45, 47, 49, 50, 52, 54, 57]
for idx in prio_indices:
    if idx < NUM_NODES: priority[idx] = 1

MAX_GLOBAL_POSITION_FOR_PRIORITY = 19 
daily_time_limits = np.array([MAX_DAILY_TIME] * N_DAYS)
daily_dist_limits = np.array([MAX_DAILY_DIST] * N_DAYS)

# ==============================
# 2. ÇÖZÜM YAPISI VE COST
# ==============================

@dataclass
class Solution:
    routes: List[List[int]]
    cost: float = None
    feasible: bool = False

def build_full_route(day_route: List[int]) -> List[int]:
    return [DEPOT] + day_route + [DEPOT]

def evaluate_solution(sol: Solution) -> Solution:
    penalty = 0.0
    hard_penalty = 0.0
    total_dist = 0.0

    # (A) Ziyaret Sayısı
    visit_counts = np.zeros(NUM_NODES, dtype=int)
    for d in range(N_DAYS):
        for node in sol.routes[d]:
            visit_counts[node] += 1

    for i in range(NUM_NODES):
        if i == DEPOT: continue
        if visit_counts[i] == 0:
            penalty += BIG_PENALTY; hard_penalty += BIG_PENALTY
        elif visit_counts[i] > 1:
            extra = BIG_PENALTY * (visit_counts[i] - 1)
            penalty += extra; hard_penalty += extra

    # Global Sıra
    global_order = []
    for d in range(N_DAYS): global_order.extend(sol.routes[d])

    # Günlük Kontroller
    for d in range(N_DAYS):
        route = sol.routes[d]
        full = build_full_route(route)
        day_dist = 0.0
        current_time = 0.0
        
        indeg = np.zeros(NUM_NODES, dtype=int)
        outdeg = np.zeros(NUM_NODES, dtype=int)

        for i in range(len(full) - 1):
            u, v = full[i], full[i + 1]
            d_uv = DIST_MATRIX[u, v]
            day_dist += d_uv
            current_time += (d_uv / AVG_SPEED)
            
            if v != DEPOT:
                a_i, b_i = time_windows[v]
                if current_time < a_i: current_time = a_i
                current_time += SERVICE_TIME
                if current_time > b_i: penalty += (current_time - b_i) * LATE_TIME_PENALTY
            
            outdeg[u] += 1; indeg[v] += 1

        total_dist += day_dist
        
        # (B) Flow & (E) Repeat
        for i in range(NUM_NODES):
            if i != DEPOT and (indeg[i] > 1 or outdeg[i] > 1):
                penalty += BIG_PENALTY; hard_penalty += BIG_PENALTY
        
        visited = [x for x in full if x != DEPOT]
        if len(visited) != len(set(visited)):
            penalty += BIG_PENALTY; hard_penalty += BIG_PENALTY

        # (D) Limitler
        if current_time > daily_time_limits[d]:
            penalty += (current_time - daily_time_limits[d]) * TIME_OVER_PENALTY
        if day_dist > daily_dist_limits[d]:
            penalty += (day_dist - daily_dist_limits[d]) * DIST_OVER_PENALTY

    # (F) Priority
    for pos, node in enumerate(global_order):
        if priority[node] == 1 and pos > MAX_GLOBAL_POSITION_FOR_PRIORITY:
            penalty += PRIORITY_PENALTY

    sol.cost = total_dist + penalty
    sol.feasible = (hard_penalty == 0.0)
    return sol

def cost_breakdown(sol: Solution):
    print("\n=== MALİYET DETAYI ===")
    print(f"Toplam Maliyet: {sol.cost:.1f}")
    print(f"Uygun (Feasible): {sol.feasible}")
    print("=== DETAY SONU ===\n")

# ==============================
# 3. ALNS OPERATÖRLERİ
# ==============================

def construct_initial_solution() -> Solution:
    nodes = list(range(NUM_NODES))
    if DEPOT in nodes: nodes.remove(DEPOT)
    random.shuffle(nodes)
    routes = [[] for _ in range(N_DAYS)]
    for idx, node in enumerate(nodes):
        routes[idx % N_DAYS].append(node)
    return evaluate_solution(Solution(routes=routes))

def random_removal(sol: Solution, q: int) -> Tuple[Solution, List[int]]:
    routes = [r[:] for r in sol.routes]
    all_nodes = [(d, i, node) for d, r in enumerate(routes) for i, node in enumerate(r)]
    if not all_nodes: return evaluate_solution(Solution(routes=routes)), []
    
    q = min(q, len(all_nodes))
    chosen = random.sample(all_nodes, q)
    chosen.sort(key=lambda x: (x[0], -x[1]))
    removed = []
    for d, i, node in chosen:
        del routes[d][i]
        removed.append(node)
    return evaluate_solution(Solution(routes=routes)), removed

def greedy_insert(sol: Solution, request_bank: List[int]) -> Solution:
    current = evaluate_solution(Solution(routes=[r[:] for r in sol.routes]))
    for u in request_bank:
        best_cost = math.inf
        best_d, best_p = 0, 0
        for d in range(N_DAYS):
            for p in range(len(current.routes[d]) + 1):
                cand_routes = [r[:] for r in current.routes]
                cand_routes[d].insert(p, u)
                cand = evaluate_solution(Solution(routes=cand_routes))
                if cand.cost < best_cost:
                    best_cost = cand.cost; best_d = d; best_p = p
        current.routes[best_d].insert(best_p, u)
        current = evaluate_solution(current)
    return current

def apply_two_opt(sol: Solution) -> Solution:
    # Basit 2-opt local search
    def route_len(r):
        full = build_full_route(r)
        return sum(DIST_MATRIX[full[i], full[i+1]] for i in range(len(full)-1))
    
    new_routes = []
    for r in sol.routes:
        best_r = r[:]
        best_l = route_len(best_r)
        improved = True
        while improved:
            improved = False
            for i in range(len(best_r)-1):
                for j in range(i+2, len(best_r)):
                    new_r = best_r[:]
                    new_r[i+1:j+1] = reversed(new_r[i+1:j+1])
                    new_l = route_len(new_r)
                    if new_l + 1e-6 < best_l:
                        best_l = new_l; best_r = new_r; improved = True; break
                if improved: break
        new_routes.append(best_r)
    return evaluate_solution(Solution(routes=new_routes))

# ==============================
# 4. ALNS FONKSİYONU
# ==============================

def alns(initial_t, alpha_t, verbose=False) -> Solution:
    S_curr = construct_initial_solution()
    S_best = S_curr
    T = initial_t
    
    for it in range(1, MAX_ITER + 1):
        if T < 1e-3: break
        
        q = max(1, (NUM_NODES - 1) // 10)
        S_tmp, bank = random_removal(S_curr, q)
        S_new = greedy_insert(S_tmp, bank)
        S_new = apply_two_opt(S_new)
        
        delta = S_new.cost - S_curr.cost
        if delta < 0 or random.random() < math.exp(-delta / T):
            S_curr = S_new
            
        if S_curr.cost < S_best.cost:
            S_best = S_curr
            
        T *= alpha_t
    return S_best

# ==============================
# 5. EXCEL KAYDETME
# ==============================

def save_final_excel(sol: Solution, runtime_sec: float, filename: str):
    rows = []
    for d, r in enumerate(sol.routes):
        for i, node in enumerate(r):
            rows.append({
                "Day": d+1, "Order": i+1,
                "University": UNIV_NAMES[node], "ID": node,
                "Priority": "Yes" if priority[node] else "No"
            })
    df_routes = pd.DataFrame(rows)
    df_summ = pd.DataFrame([{
        "Total Cost": sol.cost, "Feasible": sol.feasible,
        "Runtime (sec)": runtime_sec, "Runs": REPEAT_COUNT
    }])
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df_routes.to_excel(writer, sheet_name="Routes", index=False)
            df_summ.to_excel(writer, sheet_name="Summary", index=False)
        print(f"\n[EXCEL] Rapor kaydedildi: {filename}")
    except Exception as e:
        print(f"\n[HATA] Excel kaydedilemedi: {e}")

# ==============================
# 6. MAIN (10 TEKRAR)
# ==============================

if __name__ == "__main__":
    print(f"=== {REPEAT_COUNT} TEKRARLI FİNAL MODU ===")
    print(f"Parametreler: T={INITIAL_T}, Alpha={ALPHA_T}")
    
    global_best_sol = None
    run_costs = []
    
    total_start = time.perf_counter()

    for i in range(REPEAT_COUNT):
        current_seed = 100 + i 
        random.seed(current_seed)
        np.random.seed(current_seed)
        
        # Algoritmayı çalıştır (Logları kapattık ki ekran dolmasın)
        sol = alns(initial_t=INITIAL_T, alpha_t=ALPHA_T, verbose=False)
        
        run_costs.append(sol.cost)
        print(f" > Deneme {i+1:02d}/{REPEAT_COUNT} | Skor: {sol.cost:.1f} | Uygun: {sol.feasible}")
        
        if global_best_sol is None or sol.cost < global_best_sol.cost:
            global_best_sol = sol
    
    total_end = time.perf_counter()
    avg_time = (total_end - total_start) / REPEAT_COUNT
    
    print("\n" + "="*40)
    print("           SONUÇ ÖZETİ")
    print("="*40)
    print(f"En İyi (Min) : {min(run_costs):.1f}")
    print(f"En Kötü (Max): {max(run_costs):.1f}")
    print(f"Ortalama     : {sum(run_costs)/len(run_costs):.1f}")
    print(f"Toplam Süre  : {(total_end - total_start)/60:.1f} dk")
    
    print("\n=== ŞAMPİYON ROTA DETAYLARI ===")
    cost_breakdown(global_best_sol)
    
    for d, r in enumerate(global_best_sol.routes):
        names = [UNIV_NAMES[idx] for idx in r]
        print(f"Gün {d+1}: {names}")

    save_final_excel(global_best_sol, avg_time, REPORT_EXCEL_PATH)

[IO] Dosya okundu. Matris: (59, 59)
=== 10 TEKRARLI FİNAL MODU ===
Parametreler: T=3000.0, Alpha=0.99
ERROR! Session/line number was not unique in database. History logging moved to new session 30
 > Deneme 01/10 | Skor: 1273759.8 | Uygun: True
 > Deneme 02/10 | Skor: 1308929.0 | Uygun: True
 > Deneme 03/10 | Skor: 1290973.9 | Uygun: True
 > Deneme 04/10 | Skor: 1261739.3 | Uygun: True
 > Deneme 05/10 | Skor: 1282298.5 | Uygun: True
 > Deneme 06/10 | Skor: 1265242.7 | Uygun: True
 > Deneme 07/10 | Skor: 1288547.1 | Uygun: True
 > Deneme 08/10 | Skor: 1273088.0 | Uygun: True
 > Deneme 09/10 | Skor: 1290250.1 | Uygun: True
 > Deneme 10/10 | Skor: 1265126.1 | Uygun: True

           SONUÇ ÖZETİ
En İyi (Min) : 1261739.3
En Kötü (Max): 1308929.0
Ortalama     : 1279995.5
Toplam Süre  : 62.4 dk

=== ŞAMPİYON ROTA DETAYLARI ===

=== MALİYET DETAYI ===
Toplam Maliyet: 1261739.3
Uygun (Feasible): True
=== DETAY SONU ===

Gün 1: ['ÜSKÜDAR ÜNİVERSİTESİ', 'SAĞLIK BİLİMLERİ ÜNİVERSİTESİ', 'MARMARA Ü